In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize], axis=0)
    return np.mean(resmpld, axis=0)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    if np.abs(err)<1.0e-15:
        return str(cen)+" exact:"+str(exact)
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

In [3]:
def format_print_w_exact_matrix(exact, cen, err):
    shape=exact.shape
    res=np.full(shape, "", dtype=object)

    for i in np.arange(shape[0]):
        for j in np.arange(shape[1]):
            res[i,j]=format_print_w_exact(exact[i,j], cen[i,j], err[i,j])
            
    return np.array(res)

In [4]:
def H3x3_numeric(beta_,h_, kx_,ky_,kz_, s1,s2,s3,s4,s5,s6,s7,s8,s9):
    res = 0.0
    res += kx_*s1*s2
    res += kx_*s2*s3
    res += kx_*s3*s1
    res += kx_*s4*s5
    res += kx_*s5*s6
    res += kx_*s6*s4
    res += kx_*s7*s8
    res += kx_*s8*s9
    res += kx_*s9*s7
    #
    res += ky_*s1*s4
    res += ky_*s2*s5
    res += ky_*s3*s6
    res += ky_*s4*s7
    res += ky_*s5*s8
    res += ky_*s6*s9
    res += ky_*s7*s1
    res += ky_*s8*s2
    res += ky_*s9*s3
    #
    res += kz_*s1*s5
    res += kz_*s2*s6
    res += kz_*s3*s4
    res += kz_*s4*s8
    res += kz_*s5*s9
    res += kz_*s6*s7
    res += kz_*s7*s2
    res += kz_*s8*s3
    res += kz_*s9*s1
    
    res *= -beta_
    res -= h_*(s1+s2+s3+s4+s5+s6+s7+s8+s9)
    
    return res

In [5]:
def Z3x3_numeric(beta_,h_, kx_,ky_,kz_):
    res=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        res += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                    s1,s2,s3,s4,s5,s6,s7,s8,s9))
    return res

In [6]:
def mag_per_site3x3_numeric(beta_,h_, kx_,ky_,kz_):
    numer=0.0
    denom=0.0
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        numer += (s1+s2+s3+s4+s5+s6+s7+s8+s9)*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                                     s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))
    res = numer/denom/9.0
    return res

In [7]:
def ss_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    mat=np.array([
        [s1*s1, s1*s2, s1*s3],
        [s1*s4, s1*s5, s1*s6],
        [s1*s7, s1*s8, s1*s9]
    ])
    return mat

In [8]:
def exex_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    mat=np.array([
        [ex[0,0]*ex[0,0], ex[0,0]*ex[0,1], ex[0,0]*ex[0,2]],
        [ex[0,0]*ex[1,0], ex[0,0]*ex[1,1], ex[0,0]*ex[1,2]],
        [ex[0,0]*ex[2,0], ex[0,0]*ex[2,1], ex[0,0]*ex[2,2]]
    ])
    return mat

In [9]:
def exey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    mat=np.array([
        [ex[0,0]*ey[0,0], ex[0,0]*ey[0,1], ex[0,0]*ey[0,2]],
        [ex[0,0]*ey[1,0], ex[0,0]*ey[1,1], ex[0,0]*ey[1,2]],
        [ex[0,0]*ey[2,0], ex[0,0]*ey[2,1], ex[0,0]*ey[2,2]]
    ])
    return mat

In [10]:
def exez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ex=np.array([
        [s1*s2, s2*s3, s3*s1],
        [s4*s5, s5*s6, s6*s4],
        [s7*s8, s8*s9, s9*s7]
    ])
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ex[0,0]*ez[0,0], ex[0,0]*ez[0,1], ex[0,0]*ez[0,2]],
        [ex[0,0]*ez[1,0], ex[0,0]*ez[1,1], ex[0,0]*ez[1,2]],
        [ex[0,0]*ez[2,0], ex[0,0]*ez[2,1], ex[0,0]*ez[2,2]]
    ])
    return mat

In [11]:
def eyey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    mat=np.array([
        [ey[0,0]*ey[0,0], ey[0,0]*ey[0,1], ey[0,0]*ey[0,2]],
        [ey[0,0]*ey[1,0], ey[0,0]*ey[1,1], ey[0,0]*ey[1,2]],
        [ey[0,0]*ey[2,0], ey[0,0]*ey[2,1], ey[0,0]*ey[2,2]]
    ])
    return mat

In [12]:
def eyez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ey=np.array([
        [s1*s4, s2*s5, s3*s6],
        [s4*s7, s5*s8, s6*s9],
        [s7*s1, s8*s2, s9*s3]
    ])
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ey[0,0]*ez[0,0], ey[0,0]*ez[0,1], ey[0,0]*ez[0,2]],
        [ey[0,0]*ez[1,0], ey[0,0]*ez[1,1], ey[0,0]*ez[1,2]],
        [ey[0,0]*ez[2,0], ey[0,0]*ez[2,1], ey[0,0]*ez[2,2]]
    ])
    return mat

In [13]:
def ezez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9):
    ez=np.array([
        [s1*s5, s2*s6, s3*s4],
        [s4*s8, s5*s9, s6*s7],
        [s7*s2, s8*s3, s9*s1]
    ])
    mat=np.array([
        [ez[0,0]*ez[0,0], ez[0,0]*ez[0,1], ez[0,0]*ez[0,2]],
        [ez[0,0]*ez[1,0], ez[0,0]*ez[1,1], ez[0,0]*ez[1,2]],
        [ez[0,0]*ez[2,0], ez[0,0]*ez[2,1], ez[0,0]*ez[2,2]]
    ])
    return mat

In [14]:
def ss_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ss_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = ss_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [15]:
def exex_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exex_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exex_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [16]:
def exey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exey_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [17]:
def exez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=exez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = exez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [18]:
def eyey_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyey_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = eyey_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [19]:
def eyez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=eyez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = eyez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [20]:
def ezez_corr_numeric(beta_,h_, kx_,ky_,kz_):

    numer=ezez_matrix(0.,0.,0.,0.,0.,0.,0.,0.,0.)
    denom=0.0
    
    for s1 in [-1,1]:
        for s2 in [-1,1]:
            for s3 in [-1,1]:
                for s4 in [-1,1]:
                    for s5 in [-1,1]:
                        for s6 in [-1,1]:
                            for s7 in [-1,1]:
                                for s8 in [-1,1]:
                                    for s9 in [-1,1]:
                                        tmp = ezez_matrix(s1,s2,s3,s4,s5,s6,s7,s8,s9)
                                        numer += tmp*np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                          s1,s2,s3,s4,s5,s6,s7,s8,s9))
                                        denom += np.exp(-H3x3_numeric(beta_,h_, kx_,ky_,kz_, 
                                                                      s1,s2,s3,s4,s5,s6,s7,s8,s9))    
    res = numer/denom
    return res

In [21]:
beta=0.0746531
h=0.

kx=1.2
ky=1.5
kz=0.3

# kx=1.0
# ky=1.0
# kz=1.0

In [22]:
nin=1000
nfin=10000
nint=1
binsize=5

In [23]:
mag_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/mag/"+str(n)+".dat")
    mag_.append(dat)
    
mag=np.array(mag_)

In [24]:
jk_mag = Jackknife(mag.size, binsize)
jk_mag.set(simple_mean, mag)
jk_mag.do_it()

In [25]:
print("mag_per_site")
format_print_w_exact(0.0, jk_mag.mean(), jk_mag.err())

mag_per_site


'0.005(61) exact:0.0000 [0.08 sigma]'

In [26]:
delta=1.0e-3
h=1.0

print("mag")
numeric=(np.log(Z3x3_numeric(beta, h+delta, kx,ky,kz))-np.log(Z3x3_numeric(beta, h-delta, kx,ky,kz)))/(2.0*delta)/9.0
exact=mag_per_site3x3_numeric(beta,h, kx,ky,kz)
print(format_print_w_exact(exact, numeric, delta**2))

mag
0.88221534(100) exact:0.882215432 [0.09 sigma]


In [27]:
h=0.0

In [28]:
jk_magsq = Jackknife(mag.size, binsize)
jk_magsq.set(simple_mean, mag**2/9)
jk_magsq.do_it()

In [29]:
print("susc")
numeric=(mag_per_site3x3_numeric(beta,h+delta, kx,ky,kz)-mag_per_site3x3_numeric(beta,h-delta, kx,ky,kz))/(2.0*delta)
format_print_w_exact(numeric, jk_magsq.mean(), jk_magsq.err())

susc


'1.691(27) exact:1.6681 [0.87 sigma]'

In [30]:
# delta=1.0e-3
# print("mag")
# numeric=-(np.log(Z2x2(beta, h+delta, kx,ky,kz))-np.log(Z2x2(beta, h-delta, kx,ky,kz)))/(2.0*delta)/4.0
# exact=mag_per_site2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))
# print("susc")
# numeric=-(mag_per_site2x2(beta,h+delta, kx,ky,kz)-mag_per_site2x2(beta,h-delta, kx,ky,kz))/(2.0*delta)
# exact=susc2x2(beta,h, kx,ky,kz)
# print(format_print_w_exact(exact, numeric, delta**2))

In [31]:
ss_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ss/"+str(n)+".dat")
    ss_.append(dat)
    
ss=np.array(ss_)

jk_ss = Jackknife(ss.size, binsize)
jk_ss.set(simple_mean, ss)
jk_ss.do_it()

print(format_print_w_exact_matrix(ss_corr_numeric(beta, h, kx,ky,kz), jk_ss.mean(), jk_ss.err()))

[['1.0 exact:1.0' '0.1062(53) exact:0.10886 [0.50 sigma]'
  '0.1062(53) exact:0.10886 [0.50 sigma]']
 ['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0625(50) exact:0.05318 [1.87 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]']
 ['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]'
  '0.0625(50) exact:0.05318 [1.87 sigma]']]


In [32]:
exex_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exex/"+str(n)+".dat")
    exex_.append(dat)
    
exex=np.array(exex_)

jk_exex = Jackknife(exex.size, binsize)
jk_exex.set(simple_mean, exex)
jk_exex.do_it()

print(format_print_w_exact_matrix(exex_corr_numeric(beta, h, kx,ky,kz), jk_exex.mean(), jk_exex.err()))

[['1.0 exact:1.0' '0.1062(53) exact:0.10886 [0.50 sigma]'
  '0.1062(53) exact:0.10886 [0.50 sigma]']
 ['0.0294(41) exact:0.02812 [0.32 sigma]'
  '0.0214(39) exact:0.01672 [1.20 sigma]'
  '0.0112(39) exact:0.01807 [1.79 sigma]']
 ['0.0294(41) exact:0.02812 [0.32 sigma]'
  '0.0112(39) exact:0.01807 [1.79 sigma]'
  '0.0214(39) exact:0.01672 [1.20 sigma]']]


In [33]:
exey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exey/"+str(n)+".dat")
    exey_.append(dat)
    
exey=np.array(exey_)

jk_exey = Jackknife(exey.size, binsize)
jk_exey.set(simple_mean, exey)
jk_exey.do_it()

print(format_print_w_exact_matrix(exey_corr_numeric(beta, h, kx,ky,kz), jk_exey.mean(), jk_exey.err()))

[['0.0465(48) exact:0.03869 [1.62 sigma]'
  '0.0625(50) exact:0.05318 [1.87 sigma]'
  '0.0117(41) exact:0.01990 [2.03 sigma]']
 ['0.0183(42) exact:0.02095 [0.65 sigma]'
  '0.0227(41) exact:0.02095 [0.43 sigma]'
  '0.0155(35) exact:0.01702 [0.43 sigma]']
 ['0.0625(50) exact:0.05318 [1.87 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]'
  '0.0216(40) exact:0.01990 [0.43 sigma]']]


In [34]:
exez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/exez/"+str(n)+".dat")
    exez_.append(dat)
    
exez=np.array(exez_)

jk_exez = Jackknife(exez.size, binsize)
jk_exez.set(simple_mean, exez)
jk_exez.do_it()

print(format_print_w_exact_matrix(exez_corr_numeric(beta, h, kx,ky,kz), jk_exez.mean(), jk_exez.err()))

[['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]'
  '0.0117(41) exact:0.01990 [2.03 sigma]']
 ['0.0310(39) exact:0.02260 [2.16 sigma]'
  '0.0126(38) exact:0.01270 [0.02 sigma]'
  '0.0188(37) exact:0.01270 [1.62 sigma]']
 ['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0216(40) exact:0.01990 [0.43 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]']]


In [35]:
eyey_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyey/"+str(n)+".dat")
    eyey_.append(dat)
    
eyey=np.array(eyey_)

jk_eyey = Jackknife(eyey.size, binsize)
jk_eyey.set(simple_mean, eyey)
jk_eyey.do_it()

print(format_print_w_exact_matrix(eyey_corr_numeric(beta, h, kx,ky,kz), jk_eyey.mean(), jk_eyey.err()))

[['1.0 exact:1.0' '0.0294(41) exact:0.02812 [0.32 sigma]'
  '0.0294(41) exact:0.02812 [0.32 sigma]']
 ['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0239(38) exact:0.02161 [0.60 sigma]'
  '0.0310(39) exact:0.02260 [2.16 sigma]']
 ['0.1304(56) exact:0.13331 [0.52 sigma]'
  '0.0310(39) exact:0.02260 [2.16 sigma]'
  '0.0239(38) exact:0.02161 [0.60 sigma]']]


In [36]:
eyez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/eyez/"+str(n)+".dat")
    eyez_.append(dat)
    
eyez=np.array(eyez_)

jk_eyez = Jackknife(eyez.size, binsize)
jk_eyez.set(simple_mean, eyez)
jk_eyez.do_it()

print(format_print_w_exact_matrix(eyez_corr_numeric(beta, h, kx,ky,kz), jk_eyez.mean(), jk_eyez.err()))

[['0.1062(53) exact:0.10886 [0.50 sigma]'
  '0.0112(39) exact:0.01807 [1.79 sigma]'
  '0.1062(53) exact:0.10886 [0.50 sigma]']
 ['0.0465(48) exact:0.03869 [1.62 sigma]'
  '0.0188(37) exact:0.01270 [1.62 sigma]'
  '0.0227(41) exact:0.02095 [0.43 sigma]']
 ['0.0183(42) exact:0.02095 [0.65 sigma]'
  '0.0126(38) exact:0.01270 [0.02 sigma]'
  '0.0465(48) exact:0.03869 [1.62 sigma]']]


In [ ]:
ezez_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ezez/"+str(n)+".dat")
    ezez_.append(dat)
    
ezez=np.array(ezez_)

jk_ezez = Jackknife(ezez.size, binsize)
jk_ezez.set(simple_mean, ezez)
jk_ezez.do_it()

print(format_print_w_exact_matrix(ezez_corr_numeric(beta, h, kx,ky,kz), jk_ezez.mean(), jk_ezez.err()))